### Filter news_content

In [2]:
import pandas as pd
import re

In [4]:
# 讀入每月新聞資料內容

df1 = pd.read_json("./news/news_202301-02.json")
df3 = pd.read_json("./news/news_202303.json")
df4 = pd.read_json("./news/news_202304.json")
df5 = pd.read_json("./news/news_202305.json")
df6 = pd.read_json("./news/news_202306.json")
df7 = pd.read_json("./news/news_202307.json")
df8 = pd.read_json("./news/news_202308.json")
df9 = pd.read_json("./news/news_202309.json")
df10 = pd.read_json("./news/news_202310.json")
df11 = pd.read_json("./news/news_202311.json")
df12 = pd.read_json("./news/news_202312.json")

In [5]:
df = pd.concat([df1,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12],ignore_index=True)

In [6]:
# 過濾不需要的字詞

def remove_html_tags(text):
    html_tags_pattern = re.compile(r'<[^>]+>|&[a-zA-Z0-9]+;|\n')
    text_without_tags = html_tags_pattern.sub('', text)
    return text_without_tags

df["filtered_content"] = df.news_content.apply(remove_html_tags)

### using jieba to tokenize

In [6]:
import jieba 

In [7]:
jieba.set_dictionary('./dict.txt')
jieba.load_userdict('./dict_define.txt')

Building prefix dict from /Users/tvbs/Desktop/Python/dict.txt ...
Loading model from cache /var/folders/8x/byygknz92y9bmt68dl979xdw0000gp/T/jieba.u5ec43097e27308bdcc6412f06097ed1f.cache
Loading model cost 0.395 seconds.
Prefix dict has been built successfully.


In [4]:
del_words_path = './stops.txt'
del_words_list = []
with open(del_words_path, 'r', encoding='utf-8-sig') as f:
    for line in f.readlines():
        del_words_list.append(line.strip())

In [10]:
df["text"] = df.filtered_content.apply(jieba.lcut)

In [11]:
df.to_json("./output/tokenized_2023.json",orient="columns")

In [5]:
text = df.text
word_list = []
for sublist in text:
    filtered_sublist = [word.strip() for word in sublist if word.strip() not in del_words_list]  
    word_list.append(filtered_sublist)

In [8]:
import csv 

In [10]:
with open("./output/filtered_2023.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(word_list)  

### word embemmding  -- Dov2Vec

In [19]:
import gensim
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile

/Users/tvbs/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [20]:
for i in word_list:
    documents = [TaggedDocument(doc, [j]) for j, doc in enumerate(i)]

In [59]:
model = Doc2Vec(documents,
                    vector_size=100,
                    alpha=0.025,
                    min_alpha=0.00025,
                    min_count=5,
                    workers=4,
                    dm=1)

In [60]:
model.train(documents,total_examples=model.corpus_count,epochs=50)

In [29]:
vector = model.infer_vector(word_list[0])
print(vector)

[-3.47614265e-03 -4.29934217e-03  2.69496441e-03 -3.02094687e-03
  1.35074672e-03 -1.01084053e-03 -2.41880026e-03  4.84197075e-03
  2.91332603e-03 -4.16212482e-03 -4.02941601e-03 -3.78588127e-04
 -1.71322469e-03  9.24282649e-04  3.40163056e-03  1.00019097e-03
  1.84223056e-04  1.55916868e-03  3.88820819e-03 -3.65319429e-03
  2.64653796e-03 -4.87244409e-03 -4.73373290e-03  1.20741129e-03
  3.89830000e-03  4.03596321e-03  2.95544509e-03  2.57247081e-03
  4.94782254e-03  1.76392973e-03 -4.80976608e-03  3.85881355e-03
  5.95429563e-04  5.39318309e-04 -3.14799370e-03  1.59657840e-03
 -8.32039688e-04  3.47476779e-03  1.26299262e-03 -9.53575072e-04
  4.34962939e-03  3.47079523e-03  2.27679382e-03 -4.53739846e-03
  4.94543184e-03 -2.94528063e-03 -1.88914151e-03 -1.08825450e-03
 -2.58052349e-03  1.26856507e-03 -3.87235731e-03 -1.46435562e-03
  4.55981353e-03  1.70558633e-03 -3.08632385e-03  1.13228138e-03
  1.66258332e-03  2.73680082e-03 -2.23864801e-03 -1.86309731e-03
 -3.98293929e-03  2.02225

In [61]:
test = "民調 六都施政滿意度！盧秀燕破七成奪冠　他衝第三緊咬陳其邁" 
test = jieba.lcut(test)
test_vector = model.infer_vector(test)
model.dv.most_similar([test_vector], topn=len(model.dv))

[(64, 0.16443799436092377),
 (24, 0.13402006030082703),
 (32, 0.1264030784368515),
 (72, 0.11973859369754791),
 (56, 0.11953184008598328),
 (21, 0.1131221354007721),
 (6, 0.11261572688817978),
 (35, 0.10047830641269684),
 (73, 0.09957148134708405),
 (55, 0.09569621086120605),
 (49, 0.09443660080432892),
 (65, 0.0888744667172432),
 (16, 0.084089495241642),
 (48, 0.0828271210193634),
 (51, 0.08260615170001984),
 (38, 0.08224225789308548),
 (44, 0.08089689165353775),
 (4, 0.0720432922244072),
 (9, 0.06576717644929886),
 (1, 0.06049060821533203),
 (37, 0.05924524739384651),
 (61, 0.05786775052547455),
 (23, 0.05773860588669777),
 (20, 0.056785378605127335),
 (36, 0.0538860559463501),
 (8, 0.051988862454891205),
 (53, 0.04661101475358009),
 (22, 0.04216742515563965),
 (66, 0.03601370006799698),
 (74, 0.03453782573342323),
 (30, 0.028231332078576088),
 (39, 0.026695286855101585),
 (46, 0.02621426060795784),
 (69, 0.02611055038869381),
 (42, 0.023006131872534752),
 (62, 0.022718748077750206),

In [35]:
model.save("./output/doc2vec_2023.d2v")

### word embemmding  -- Word2Vec

In [70]:
word2vec = Word2Vec(word_list, vector_size=250, epochs=10, sg=1)

In [71]:
word2vec.save("./output/word2vec_2023.w2v")

In [73]:
import numpy as np

In [82]:
word = "農曆春節天氣冷颼颼，受強烈大陸冷氣團、輻射冷卻影響，中央氣象署指出，儘管明（11）日大年初二多雲到晴，北部和東北部依舊寒冷，其他地區早晚也陷入低溫。尤其西半部日夜溫差甚大，恐怕跌破8度，而這波影響也將持續到初三清晨，提醒民眾時刻注意保暖。"
model = word2vec
def calculate_text_feature(word, model):

    # 初始化特徵向量
    text_feature = np.zeros((model.vector_size,), dtype=np.float32)
    # 計算每個詞的嵌入式向量並加總
    for word in word:
        if word in model.wv:
            text_feature += model.wv[word]
    # 取平均
    text_feature /= len(word)
    return text_feature

# 計算文本的特徵向量
feature_vector = calculate_text_feature(word, model)
print("文本的特徵向量:", feature_vector)

文本的特徵向量: [-3.55864525e+00  7.57046795e+00  4.11416435e+00 -7.19501066e+00
 -8.54550076e+00 -1.70852890e+01  1.13265104e+01 -1.29841423e+01
  3.22108459e+00 -6.18759108e+00 -4.50612116e+00  2.10676742e+00
 -9.46172619e+00 -1.44602785e+01  3.58402014e+00 -1.27542710e+00
  1.73780270e+01  6.77771521e+00 -8.19639015e+00 -1.01580763e+01
 -7.58840513e+00  1.80167046e+01  1.36039314e+01  1.58508692e+01
 -6.29196882e+00  3.32420397e+00  1.68356371e+00  3.71187806e-01
 -5.94329071e+00 -8.32559586e+00  1.92586327e+01  8.09424210e+00
  4.79435921e-03  1.29425454e+00 -3.04595470e-01  1.08792343e+01
  2.18994904e+01 -7.10358620e+00 -6.73931932e+00 -7.73047256e+00
 -6.85317898e+00 -9.47985172e+00  1.55379906e+01 -1.33466072e+01
 -6.53804421e-01  1.14518394e+01  1.83512058e+01  6.82422280e-01
 -1.47229652e+01  7.07719231e+00  3.26442289e+00  5.65034866e+00
  3.68283582e+00 -1.58790503e+01  5.36352634e+00  8.34086323e+00
  1.11622000e+00  5.94711876e+00 -2.52697983e+01 -1.55839806e+01
 -2.34279060e+00